In [80]:
import simpy 
import random
import math

In [81]:
#Numero de Rampas y Fitters. Ademas se define a joe como recurso (igual a 1)
ramps = 5
fitters = 4
joes = 1

In [82]:
# Tiempo de simulacion y demas constantes/variables (en minutos)
#Maximo 4 carros en fila o se van los que llegan
tiemposimulacion = 600
maxcolainspeccion = 4
colainspeccion = 0
tiempocolaInspeccion = 0
averagetiempoperclient = 0
tottim = 0
totaltiempo = 0
clientesPostinspeccion = 0
clientesPostServicio = 0
totalclientes = 0

In [83]:
#Tiempo de Inspeccion: Distribucion normal con media 5 y desviacion estandar de 1.5 (minutos)
def inspeccion(env, cliente):
    Tiempo = random.normalvariate(5,1.5)
    yield env.timeout(Tiempo)
    print(cliente + ' inspeccion: %3.f minutos.' %(Tiempo))

#Tiempo de fitting: Distribucion lognormal con media 40 y desviacion estandar 10 (minutos)
def fitting(env, cliente):
    Tiempo = random.normalvariate(40,10)
    yield env.timeout(Tiempo)
    print(cliente + ' fitting: %3.f minutos.' %(Tiempo))

#Segunda inspeccion (Por parte de Joe)
def secondinspeccion(env, cliente):
    Tiempo = random.uniform(1,3)
    yield env.timeout(Tiempo)
    print(cliente + ' segunda inspeccion: %3.f minutos.' %(Tiempo))

#Tiempo de pago: Distribucion exponencial media de 3 minutos
def pago(env, cliente):
    Tiempo = random.expovariate(1/3)
    yield env.timeout(Tiempo)
    print(cliente + ' pago: %3.f minutos.' %(Tiempo))

In [84]:
#Generacion de la llegada de los clientes al parlour
#Distribucion exponencial con media de 15 minutos
def llegadasClientes (env, ramp, fitter, joe):
  cont = 0
  while True:
    arrivalTiempo = random.expovariate(1/15)
    yield env.timeout(arrivalTiempo)
    cont+=1
    client= 'Cliente ' + str(cont)
    env.process(customers(env, client, ramp, fitter, joe))

In [85]:
# Metodo global, toma el environment y los recursos del problema
def customers(env, cliente, ramp, fitter, joe):
  global simulationTiempo,tottim, totalTiempo, clientesPostinspeccion, clientesPostServicio, colainspeccion, maxcolainspeccion, totalclientes

#Tiempo de llegada del clientes
  arrivalTiempo=env.now
  totalclientes += 1
  print(cliente +' llega: minuto %.2f .' %(arrivalTiempo)) 
  
#Solo entra a la fila si hay menos de 4 carros, si no simplemente de va del parlour
  if colainspeccion >= maxcolainspeccion:
    print(cliente + ' se va. Fila llena:  %.2f minutos' %(env.now))
  else:
# Entra al parlour en donde utiliza una rampa y a Joe
# En caso de haber fila, hace fila y luego pasa a la inspeccion
    colainspeccion += 1
    with ramp.request() as useRamp:
      yield useRamp
      with joe.request() as useJoe:
        yield useJoe
        colainspeccion = colainspeccion - 1
        esperainspeccionTiempo = env.now - arrivalTiempo
        if esperainspeccionTiempo == 0:
          print(cliente + ' inspeccion: sin fila')
        else:
          print(cliente + ' inspeccion: %.2f minutos en fila. <-----------------' %(esperainspeccionTiempo))
        yield env.process(inspeccion(env, cliente))
        joe.release(useJoe)
      ramp.release(useRamp)
    #Tiempo final de la primera inspeccion
    endOfinspeccionTiempo = env.now

#Luego de dar el consejo inicial por parte de Joe, el 30% de los clientes se van y el 70% se quedan
#Si hace parte del 70% para al fitting donde utiliza una rampa y un fitter (Joe ya esta libre aca)
    abandon = random.uniform(0,1)
    if abandon < 0.7:
      while True:
        with ramp.request() as useRamp:
          yield useRamp
          with fitter.request() as useFitter:
            yield useFitter
            esperacolaTiempoForFitting = env.now - endOfinspeccionTiempo
            if esperacolaTiempoForFitting == 0:
              print( cliente +' fitting: sin fila')
            else:
              print(cliente + ' fitting: %.2f minutos en fila.' %(esperacolaTiempoForFitting))
            yield env.process(fitting(env, cliente))
            fitter.release(useFitter)
          endOfFittingTiempo = env.now
#Tiempo final de fitting
#Se utiliza a joe para ls "Segunda" Inspeccion
          with joe.request() as useJoe:
            yield useJoe
            esperacolaTiempoForinspeccion = env.now - endOfFittingTiempo
            if esperacolaTiempoForinspeccion == 0:
                  print( cliente +' inspeccion: sin fila')
            else:
                  print(cliente + ' segunda inspeccion: %.2f minutos en fila.' %(esperacolaTiempoForinspeccion))
            yield env.process(secondinspeccion(env, cliente))
            joe.release(useJoe)
          ramp.release(useFitter)
#10% de las veces, el trabajo no es satisfactorio y debe de hacerse un refitting. 
#Si esta en el 90% de los clientees simplemente pasa al pago
          refitting = random.uniform(0,1)
          if refitting <0.9:
            break
          else:
            print(cliente+' tiene que hacer refitting.')
    
      endTiempo = env.now
#Se utiliza a Joe para el pago luego de pasar por las inspecciones y el fitting
      with joe.request() as useJoe:
        yield useJoe
        esperacolaTiempoForpago = env.now - endTiempo
        if esperacolaTiempoForpago == 0:
              print( cliente +' pago: sin fila')
        else:
              print(cliente + ' pago: %.2f minutos en fla.' %(esperacolaTiempoForpago))
        yield env.process(pago(env, cliente))
        joe.release(useJoe)  

#El clientee termina el ciclo y sale del parlour
      exitTiempo = env.now
      totalTiempo = exitTiempo - arrivalTiempo
      print(cliente +' acaba sesion en parlour: minuto %.2f' %(env.now))
      print('----------------------------------> Total Tiempo %.2f' %(totalTiempo))
      clientesPostServicio += 1
      tottim += totalTiempo

    else: 
    #Aca es si abandono despues de la primera inspeccion (30%)
      print(cliente + ' abandona el parlour: minuto %.2f' %(env.now))
      clientesPostinspeccion += 1




In [86]:
#De aca corremos el proceso de llega de clientes y todo el enviroment con el tiempo de simulacion y los recursos definidos
def main():
  print('Start of simulation')
  env = simpy.Environment()
  a = 0.2 #choose a good random seed
  random.seed(a)
#Definicion de los recursos
  fitter = simpy.Resource(env, fitters)
  ramp = simpy.Resource(env, ramps)
  joe = simpy.Resource(env, joes)
  env.process(llegadasClientes(env, ramp, fitter, joe))
  env.run(until = tiemposimulacion)


In [87]:
main()

Start of simulation
Cliente 1 llega: minuto 3.13 .
Cliente 1 inspeccion: sin fila
Cliente 2 llega: minuto 3.27 .
Cliente 3 llega: minuto 7.94 .
Cliente 1 inspeccion:   5 minutos.
Cliente 1 abandona el parlour: minuto 8.14
Cliente 2 inspeccion: 4.88 minutos en fila. <-----------------
Cliente 2 inspeccion:   5 minutos.
Cliente 2 abandona el parlour: minuto 13.05
Cliente 3 inspeccion: 5.11 minutos en fila. <-----------------
Cliente 3 inspeccion:   5 minutos.
Cliente 3 fitting: sin fila
Cliente 4 llega: minuto 22.68 .
Cliente 4 inspeccion: sin fila
Cliente 5 llega: minuto 23.77 .
Cliente 4 inspeccion:   5 minutos.
Cliente 5 inspeccion: 3.55 minutos en fila. <-----------------
Cliente 4 fitting: sin fila
Cliente 5 inspeccion:   4 minutos.
Cliente 5 fitting: sin fila
Cliente 6 llega: minuto 33.70 .
Cliente 6 inspeccion: sin fila
Cliente 6 inspeccion:   4 minutos.
Cliente 6 fitting: sin fila
Cliente 5 fitting:  10 minutos.
Cliente 5 inspeccion: sin fila
Cliente 3 fitting:  23 minutos.
Clien

In [90]:
def stats1():
  global tottim
  print('Stats')
  averagetimeperclient = tottim / clientesPostServicio
  print('Tiempo total promedio por cliente: %.3f' %(averagetimeperclient))

In [91]:
stats1()

Stats
Tiempo total promedio por cliente: 57.115


In [79]:
mu = 40
sigma = 10
  
# using the lognormvariate() method
print(random.lognormvariate(mu, sigma))

2640387.024874524
